In [ ]:
import os 
import shutil
import zipfile
from tqdm.auto import tqdm
is_kaggle = os.path.exists('/kaggle')

if is_kaggle and not os.path.exists('./images_tags_final/'):
    os.makedirs('images_tags_final',exist_ok=True)
    if os.path.exists('/kaggle/input/sdchairs/images_tags_final.zip'):
        with zipfile.ZipFile('/kaggle/input/sdchairs/images_tags_final/'+file, 'r') as zip_ref:
            zip_ref.extractall('./images_tags_final/')
    else:
        p = '/kaggle/input/sdchairs/images_tags_final' 
        if os.path.exists('/kaggle/input/sdchairs/images_tags_final/images_tags_final'):
            p += '/images_tags_final/'
      
        for file in tqdm(os.listdir(p)):
            shutil.copy(p+file,'./images_tags_final/'+file)

        shutil.copy('/kaggle/input/sdchairs/meta_cap_buckets.json','./meta_cap_buckets.json')

import json 
data = json.load(open('./meta_cap_buckets.json','rb'))
replace_parent_dirs = list(data.keys())[0].split('images_tags_final')[0]
replace_parent_dirs = replace_parent_dirs[:-1] 
#os.path.abspath('.'), replace_parent_dirs
new_data = {}
for k, v in data.items():
    new_data[k.replace(replace_parent_dirs,os.path.abspath('.'))] = v

with open('./meta_cap_buckets.json','w') as f:
    json.dump(new_data, f)

In [ ]:
import os 

image_dir = './images_tags_final' 


#metadata_file = 'C:\piyo\piyo_md.json'

epochs = 7
your_number_of_unique_images = 5234
resolution = 512
batch_size = 1
num_repeats = 1
gradient_accumulation_steps = 2

temp_n_updates_per_epoch = (your_number_of_unique_images*num_repeats)//(gradient_accumulation_steps*batch_size)
max_train_steps = epochs * temp_n_updates_per_epoch
#max_bucket_reso = 512
use_mixed_precision = 'fp16'
lr = 3e-6



flip_aug = True
toml = f"""[general]
shuffle_caption = true
caption_extension = '.txt'
keep_tokens = 1

[[datasets]]
#resolution = [{resolution}, {resolution}]
batch_size = {batch_size}
enable_bucket= true
max_bucket_reso = 1024
min_bucket_reso = 128
bucket_reso_steps = 64

    [[datasets.subsets]]
    image_dir = '{image_dir}'
    metadata_file = '{image_dir}'
    keep_tokens = 1
    flip_aug = true
    num_repeats = {num_repeats}
    

"""
# check if image_dir exists
if not os.path.exists(image_dir):
    raise Exception(f"Image directory {image_dir} does not exist")
# make it full path
image_dir = os.path.abspath(image_dir)

with open('dataset_config.toml', 'w') as f:
    f.write(toml)

dataset_toml = os.path.abspath('dataset_config.toml')

In [ ]:
# Sample prompts
# 1 von fff852e4-140f-53d4-a9be-7f4c6558abf0.jpg
# 2 von fa257c34-6d93-5f57-9b2c-579d8b59890f.jpg
training_sample_prompts = """
Preis 150, Allgemeine Angaben Breite: 66cm, Allgemeine Angaben Hoehe: 98cm, Allgemeine Angaben Tiefe: 66cm, Allgemeine Angaben Gewicht 11kg, Allgemeine Angaben Farbe Dunkelgrau, Technische Details Max. Belastbarkeit 136kg, Produktmerkmale Rueckenlehne Hoehe 49cm, Armlehnen/Armteil Armlehnenhoehe 18cm, Bezug Material Textil, Material Textil, Material Oberflaeche Textil Textilgeflecht, Kundenhinweise Pflegehinweise Keine scharfen Reinigungsmittel verwenden --w 512 --h 512 --d 1 --l 7.5 --s 28

Preis 370, Allgemeine Angaben Breite: 56cm, Allgemeine Angaben Hoehe: 86cm, Allgemeine Angaben Tiefe: 59cm, Allgemeine Angaben Gewicht 15kg, Allgemeine Angaben Farbe Braun, Artikel besteht aus Anzahl der Stuehle 2, Technische Details Anzahl der Sitzplaetze 1, Produktmerkmale Bezug Stoffzusammensetzung 100% Polyester, Material Holzwerkstoff --w 512 --h 512 --d 1 --l 7.5 --s 28

Preis 499, Allgemeine Angaben Breite: 62cm, Allgemeine Angaben Hoehe: 90cm, Allgemeine Angaben Tiefe: 65cm, Allgemeine Angaben Farbe Haselnussfarben, Allgemeine Angaben Farbe Cognac, Allgemeine Angaben Farbe Wildeiche, Produktvorteile Zubehoer inklusive Armlehnen, Produktvorteile Gestell, Produktvorteile Besonderheiten Lederauswahl, Produktvorteile Stoffauswahl, Produktvorteile Typenauswahl, Produktvorteile inklusive Armlehnen, Produktvorteile Gestellauswahl, Artikel besteht aus Anzahl der Rueckenkissen 1, Artikel besteht aus Rueckenkissen, Technische Details Max. Belastbarkeit 140kg, Armlehnen/Armteil Armlehnenhoehe 66cm, Material Oberflaeche geoelt, Material Lederart Kombination Echtleder/Stoff, Material Lederqualitaet Rindleder, Material Holzart Eiche, Material Holzqualitaet Echtholz, Material Echtholz massiv, Material Scheuertouren 50.000, Material Oberflaeche Leder pigmentiert, Material Lichtechtheit 5 gut, Material Abriebfestigkeit 4 gut, Material Pillingbildung 4 leicht, Material Holz, Material Textil, Material Leder, Material Oberflaeche Textil Webstoff, Material Materialzusammensetzung 100 % Polyester, Beine Farbe Eiche Bianco, Beine Anzahl der Beine 4, Beine Beinform konisch, Beine Stativgestell, Sitzflaeche Sitzbreite 42-47cm, Sitzflaeche Sitzhoehe 48cm, Sitzflaeche Sitztiefe 42cm, Innenleben Rueckenlehne Schaumstoff, Innenleben Schaumstoff, Kundenhinweise Verwendung Indoor, Kundenhinweise Lieferhinweis fuer Endkunden zerlegt, Kundenhinweise Jedes Echtholzprodukt ist ein Unikat. Woelbungen, Kundenhinweise Farbverlaeufe etc. sind warentypisch und stellen keinen Reklamationsgrund dar., Kundenhinweise Transporthinweis fuer Endkunden Kombi, Kundenhinweise Montagehinweis fuer Endkunden handwerkliche Begabung, Kundenhinweise Pflegehinweise Holzpflegemittel verwenden, Kundenhinweise Lederpflegemittel verwenden, Kundenhinweise nebelfeucht wischen, Verpackungsmasse Anzahl der Pakete 2, Verpackungsmasse Paket 1 Laenge: 65cm, Verpackungsmasse Paket 1 Breite: 65cm, Verpackungsmasse Paket 1 Hoehe: 76cm, Verpackungsmasse Paket 2 Laenge: 56cm, Verpackungsmasse Paket 2 Breite: 51cm --w 512 --h 512 --d 1 --l 7.5 --s 28
"""
# d is random seed, l classifier free guidance, s number of steps

# display images
from PIL import Image
import matplotlib.pyplot as plt

image_paths = [
    'images_tags_final/fff852e4-140f-53d4-a9be-7f4c6558abf0_149.90_2.jpg',
    'images_tags_final/fa257c34-6d93-5f57-9b2c-579d8b59890f_369.95_2.jpg',
    'images_tags_final/de28ad4c-0927-5ebd-b31c-2636420ac44a_499._1.jpg'
]

#fig, axs = plt.subplots(1, len(image_paths), figsize=(10, 5))

#for i, path in enumerate(image_paths):
#    img = Image.open(path)
#    axs[i].imshow(img)
#    axs[i].axis('off')

#plt.show()


### Prepare kohya sd-script specific setup


In [ ]:
import torch 
import os 
if not os.path.exists('sd-scripts'):
    !git clone https://github.com/kohya-ss/sd-scripts
    # if mps is available, install those requirements
    if torch.backends.mps.is_available():
        print('Macbook installation')
        !git clone --recursive https://github.com/bmaltais/kohya_ss -q
        os.chdir('kohya_ss')
        !pip install xformers bitsandbytes==0.41.1 -q
        !pip install tensorflow-macos tensorflow-metal tensorboard==2.14.1 -q
        !pip install onnxruntime==1.17.1
        !pip install -r requirements.txt -q
        os.chdir('..')
        # fix for xformers macos https://github.com/bmaltais/kohya_ss/issues/2071
        !ls /Users/amos/miniconda3/envs/sd-scripts/lib/python3.10/site-packages -q
        !rm -r /Users/amos/miniconda3/envs/sd-scripts/lib/python3.10/site-packages/xformers -q
        !rm -r /Users/amos/miniconda3/envs/sd-scripts/lib/python3.10/site-packages/xformers-0.0.26.post1.dist-info -q
    else: 

        os.chdir('sd-scripts')

        ! pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118 -q
        !pip install opencv-python>=4.9.0
        !pip install huggingface-hub>=0.21.2
        !pip install -r requirements.txt -q
        !pip install xformers==0.0.23.post1 --index-url https://download.pytorch.org/whl/cu118 -q
        



        os.chdir('..')

In [ ]:
# change directory to the sd-scripts folder
if not is_kaggle:
    os.chdir('sd-scripts')
    # make training_metadata_tags.json file 
    !
    command = f"python finetune/merge_dd_tags_to_metadata.py --full_path {image_dir} meta_cap.json"
    os.system(command)

    !python finetune/clean_captions_and_tags.py meta_cap.json meta_cap_clean.json
    os.chdir('..')

    metadata_file_path = os.path.abspath('sd-scripts/meta_cap_clean.json')

## Training
Only the UNet is finetuned by default, the text encoder is left as-is.
The base model used is https://huggingface.co/runwayml/stable-diffusion-v1-5.

In [ ]:
resume_path = None
if not is_kaggle and False:
    from huggingface_hub import hf_hub_download
    import os
    # import shutil

    cache_dir = '.cache'
    os.makedirs(cache_dir, exist_ok=True)
    # amos: issues with symlinks (rather just download manually, put in folder and specify link manually)
    model = hf_hub_download(repo_id="runwayml/stable-diffusion-v1-5", filename="v1-5-pruned.safetensors" )
    model_path = os.path.abspath(model)
else:
    model_path = '/kaggle/input/sdchairs/downloaded_models/v1-5-pruned.safetensors'
    if os.path.exists('/kaggle/input/sdchairs/checkpoint/'):
        print('use checkpoint')
        #model_path = '/kaggle/input/sdchairs/checkpoint/'
        #model_path = '/kaggle/input/sdchairs/checkpoint/model.safetensors'
        resume_path = '/kaggle/input/sdchairs/checkpoint/'


In [ ]:
if not is_kaggle:
    !pip install torch==2.1.2 # fix "torch has no attribute mps"

#else:
    

In [ ]:
if not is_kaggle:
    # precompute latents (using encoder of the default model)
    metadata_file_path = os.path.abspath('sd-scripts/meta_cap_clean.json')

    os.chdir('sd-scripts')
    command = f'python finetune/prepare_buckets_latents.py --full_path {image_dir} {metadata_file_path} meta_cap_buckets.json "{model}" --batch_size {batch_size} --max_resolution {max_bucket_reso},{max_bucket_reso} --mixed_precision {use_mixed_precision} --max_data_loader_n_workers=4'
    os.system(command)
    os.chdir('..')
    metadata_file_path = os.path.abspath('sd-scripts/meta_cap_buckets.json')
else:
    metadata_file_path = '/kaggle/working/meta_cap_buckets.json'

In [ ]:
# this code is adapted from https://github.com/Linaqruf/kohya-trainer/
import datetime
import toml
import os
import torch

%store -r

#model_path = '/kaggle/input/sdchairs/downloaded_models/v1-5-pruned.safetensors' #'/kaggle/working/sd-scripts/checkpoints/chair_2024-06-13_14-37-10-000001.safetensors' 
#resume_path = None #'/kaggle/working/sd-scripts/checkpoints/chair_2024-06-13_14-37-10-000001-state'

dataset_repeats = num_repeats #
output_dir = 'checkpoints'
# = None #'/kaggle/working/sd-scripts/checkpoints/' #None
#os.makedirs(output_dir, exist_ok=True)

output_filename = f"chair_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

enable_sample_prompt = True  # @param {type:"boolean"}
sampler = "ddim"  # @param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
noise_offset = 0.0  # @param {type:"number"}
# specified above max_train_steps = 10000  # @param {type:"number"}
#lr_warmup_steps = 512
train_batch_size = batch_size  # @param {type:"number"}
mixed_precision = 'fp16'  # @param ["no","fp16","bf16"] {allow-input: false}
# save_state = False  # @param {type:"boolean"}
save_precision = "fp16"  # @param ["float", "fp16", "bf16"] {allow-input: false}
#save_n_epoch_ratio = 0.2  # @param {type:"number"}
save_every_n_epochs = 7
save_model_as = "safetensors"  # @param ["ckpt", "safetensors", "diffusers", "diffusers_safetensors"] {allow-input: false}
max_token_length = 225  # @param {type:"number"}
clip_skip = 2  # @param {type:"number"}
gradient_checkpointing = True  # @param {type:"boolean"}
# soecufued above gradient_accumulation_steps = 1  # @param {type:"number"}
seed = -1  # @param {type:"number"}
logging_dir = "fine_tune/logs"
prior_loss_weight = 1.0

# os.chdir(repo_dir)

# sample_str = f"""
#   masterpiece, best quality, 1girl, aqua eyes, baseball cap, blonde hair, closed mouth, earrings, green background, hat, hoop earrings, jewelry, looking at viewer, shirt, short hair, simple background, solo, upper body, yellow shirt \
#   --n lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry \
#   --w 512 \
#   --h 768 \
#   --l 7 \
#   --s 28
# """

config = {
    "model_arguments": {
        "v2": False,
        # "v_parameterization": v_parameterization if v2 and v_parameterization else False,
        "v_parameterization": False,
        "pretrained_model_name_or_path": model_path,
        # "vae": vae,
    },
    "optimizer_arguments": {
        # "min_snr_gamma": min_snr_gamma if not min_snr_gamma == -1 else None,
        "optimizer_type": 'AdamW8bit',
        "learning_rate": lr,
        # "max_grad_norm": 1.0,
        # "train_text_encoder": train_text_encoder,
        # "optimizer_args": eval(optimizer_args) if optimizer_args else None,
        # "lr_scheduler": lr_scheduler,
        #"lr_warmup_steps": lr_warmup_steps,
        # "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        # "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
    },
    "dataset_arguments": {
        "debug_dataset": False,
        "in_json": metadata_file_path,
        "train_data_dir": image_dir,
        "dataset_repeats": dataset_repeats,
        "shuffle_caption": True,
        # "keep_tokens": keep_tokens, # ?
        "resolution": f"{resolution},{resolution}",
        "caption_dropout_rate": 0,
        "caption_tag_dropout_rate": 0,
        "caption_dropout_every_n_epochs": 0,
        "color_aug": False,
        "flip_aug": flip_aug, # amos
        "face_crop_aug_range": None,
        "token_warmup_min": 1,
        "token_warmup_step": 0,
    },
    "training_arguments": {
        "output_dir": output_dir,
        "output_name": output_filename,
        "save_precision": save_precision,
        "save_every_n_epochs": save_every_n_epochs,
        #"save_n_epoch_ratio": save_n_epoch_ratio,
        "save_last_n_epochs": None,
        "save_state": True,
        "save_last_n_epochs_state": None, # only save last state ?
        "resume": resume_path,
        "train_batch_size": train_batch_size,
        "max_token_length": 225,
        "mem_eff_attn": False,
        "xformers": False if not torch.backends.mps.is_available() else False, #change to true again
        "max_train_steps": max_train_steps, 
        "max_data_loader_n_workers": 4,
        "persistent_data_loader_workers": True,
        "seed": seed if seed > 0 else None,
        "gradient_checkpointing": gradient_checkpointing,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "mixed_precision": mixed_precision,
        # "clip_skip": clip_skip if not v2 else None,
        "logging_dir": logging_dir,
        "log_prefix": output_filename,
        "noise_offset": noise_offset if noise_offset > 0 else None,
    },
    "sample_prompt_arguments": {
        #"sample_every_n_steps": 100 if enable_sample_prompt else 999999,
        "sample_every_n_epochs": 1,
        "sample_sampler": sampler,
    },
    "saving_arguments": {
        "save_model_as": save_model_as
    },
}

config_path = os.path.abspath("config_file.toml")
prompt_path = os.path.abspath("sample_prompt.txt")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

write_file(config_path, config_str)
write_file(prompt_path, training_sample_prompts)

print(config_str)

In [ ]:
model_path

In [ ]:
#if os.path.abspath('') == '/kaggle/working':
#    os.makedirs('./download')
#    for file in os.listdir('./sd-scripts/checkpoints/sample/'):
#        shutil.copy('./sd-scripts/checkpoints/sample/'+file,'./download/'+file)

In [ ]:
import datetime 
no_gpus = 1
accelerate_config = 'accelerate_config.toml'
accelerate_config = os.path.abspath(accelerate_config)
from accelerate.utils import write_basic_config

if not os.path.exists(accelerate_config):
    write_basic_config(save_location=accelerate_config)

#output_dir = 'checkpoints'
#os.makedirs(output_dir, exist_ok=True)
#output_filename = f"chair_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

os.chdir('sd-scripts')
command = f'''accelerate launch --num_processes={no_gpus} --config_file={accelerate_config} --num_cpu_threads_per_process=1 fine_tune.py --sample_prompts={prompt_path} --config_file={config_path}'''
os.system(command)
os.chdir('..')

In [ ]:
!zip ./sample.zip /kaggle/working/sd-scripts/checkpoints/sample/*

In [ ]:
exit()